# Hands-On 1: Audio Feature Extraction (Group. H)


For this first hands-on session, we are going to investigate the extraction of audio features. This is the first step for designing a classification model (in future hands-on sessions). <br>
As the recording of audio signals using a microphone will also be covered in a future hands-on session, we will start here by using sounds that we recorder for you. They have been placed in the `soundfiles` subdirectory.

To ensure you are catching the content of this notebook, we leave you with an infinitesimal amount of **code to write**. 

You will find the zones to be briefly filled  with a `### TO COMPLETE` in the cells below.

In [ ]:
import librosa  # For audio signal computations as MFCC
import matplotlib.pyplot as plt
import numpy as np
import sounddevice as sd
import soundfile as sf
from scipy import signal
from scipy.fftpack import dct

from classification.datasets import Dataset, get_cls_from_path
from classification.utils.plots import plot_audio, plot_specgram

We then create a matrix with path names of height H=5 classes and W=20 sounds per class. This will give you simple access to any sound from the dataset.

In [ ]:
dataset = Dataset()

print("\n".join(dataset.list_classes()))

You can now select a sound from a given class using ``dataset[class_name, sound_index]``. For example, the first sound of the ``chainsaw`` class is accessed with ``dataset["chainsaw", 0]`` and the following cell plays the sound:

In [ ]:
sound = dataset["fire", 0]
x, fs = sf.read(sound)
target_dB = 25
x /= np.linalg.norm(x) * 10 ** (-target_dB / 20)
print(f'Playing a "{get_cls_from_path(sound)}"')
#sd.play(x, fs)

We now ask you to complete the cells below.

## 1) Resampling and filtering

Most probably your circuit board will sample the analog audio signal at a frequency $f_s = 11025$ Hz. <br>
However, the audio provided in the dataset are sampled with $f_s = 44100$ Hz, you should thus downsample each audio signal to keep coherency with your real setup. There are 2 solutions:
- Rewrite a new dataset with the downsampled audio signals.
- Downsample each audio which is read.

We provide you with the second one.

***
#### <u> The following derivations are not necessary for the rest of this notebook, but are still provided for the curious students... </u>

Let us consider one original audio signal from the dataset and denote it $x[n]$, for $n=0,\dots,N-1$.

The downsampled signal $y$ can be written as 

$$
    y[m] = w[mM],\quad \text{with}\ w[k] = (h \ast x)[k] = \sum_{n=-\infty}^{\infty} h[n]x[k-n],
$$

where $h$ is a discrete low-pass filter and $M$ is the downsampling factor, here $M=4$. <br>

We can expand both $y$ and $w$ according to their Fourier series (DTFT) $Y$ and $W$, respectively, so that:

$$
    y[m] = \frac{1}{2\pi} \int_0^{2\pi} Y(e^{j\Omega}) e^{jm\Omega} d\Omega \tag{1}
$$
$$
    w[mM] = \frac{1}{2\pi} \int_{0}^{2\pi} W(e^{j\Omega}) e^{jmM\Omega} d\Omega = \frac{1}{2\pi} \sum_{k=0}^{M-1} \int_{2\pi k/M}^{2\pi(k+1)/M} W(e^{j\Omega}) e^{jmM\Omega} d\Omega \tag{2}
    $$

Regarding $w$, applying the change of variable $\Omega \leftarrow \Omega-2\pi k/M$ to each integral of the sum, and changing $k \leftarrow M - k$, we can further write

$$
   \textstyle w[mM] = \frac{1}{2\pi} \int_0^{2\pi/M} \sum_{k=0}^{M-1} W[e^{j(\Omega - 2\pi k/M)}] e^{jmM\Omega} d\Omega.
$$


With a final change of variable $\Omega \leftarrow M\Omega$, we get
$$
    \textstyle w[mM] = \frac{1}{2\pi} \int_0^{2\pi} \frac{1}{M} \sum_{k=0}^{M-1} W[e^{j(\Omega - 2\pi k)/M}] e^{jm\Omega} d\Omega.
$$

And by identifying (1) and (2), this yields:
$$
    Y(e^{j\Omega}) = \frac{1}{M} \sum_{k=0}^{M-1} W[e^{j(\Omega -2\pi k)/M}]
$$

***

In practice, here is the observed phenomenon when downsampling the spectrum above with factor $M=2$.

<center> <img src="figs/downsampling.png" alt=""  width="600" height="300"/> </center>

In [ ]:
fs_down = 11025  # Target sampling frequency
sound = dataset["birds", 25]  # Sound choice (default should be Handsaw)

print("Playing and showing data for : ", get_cls_from_path(sound))
x, fs = sf.read(sound)


M = fs // fs_down  # Downsampling factor
print("Downsampling factor: ", M)

### TO COMPLETE
### Downsample "audio"
x_naive_down = x[::M]

plot_audio(x, x_naive_down, fs, fs_down)  # Function call

Is your downsampling working properly? What can you observe on the spectrum of the downsampled signal? How is this phenomenon named and what is its origin?

In order to avoid it, the original signal should be low-pass filtered prior to downsampling (as presented in the mathematics above).

In [ ]:

"Low-pass filtering before downsampling"
N = 100  # number of taps
taps = signal.firwin(numtaps=N, cutoff=fs_down / 2, window="hamming", fs=fs)
x_filt = np.convolve(x, taps, mode="full")

### TO COMPLETE
### Downsample ``audio_filt``
x_filt_down = x_filt[::M]

plot_audio(x, x_filt_down, fs, fs_down)

The obtained spectrum of the downsampled signal should not suffer from aliasing anymore. In fact, there is a built-in function in ``scipy.signal`` that performs the downsampling, including a low-pass filter: ``scipy.signal.resample``. Its docstring is:

In [ ]:
help(signal.resample)

In the following, we use this function.

In [ ]:
y = signal.resample(x, int(len(x) / M))
L = len(y)

plot_audio(x, y, fs, fs_down)

Can you hear the differences between the downsampled versions of the audio signal and the original one?

In [ ]:
target_dB = 25
x /= np.linalg.norm(x) * 10 ** (-target_dB / 20)
#sd.play(x, fs, blocksize=1024)
#sd.play(x_naive_down, fs_down, blocksize=1024)
#sd.play(x_filt_down, fs_down, blocksize=1024)
#sd.play(y, fs_down, blocksize=1024)

You can also try sounds from different classes by running again the code above and changing the choice of the variable ``sound``. Maybe you will notice there are two *original* and two *downsampled* signals shown. Can you guess why you have two signals? How to deal with this?

Now we are working with sound signals with same sampling frequency as for the project, we can go on.

## 2) Windowing and spectrogram computation

A very intuitive way to represent an audio signal is with a time-frequency analysis. 
The spectrogram of a signal consists in applying an FFT on successive subpieces of it, and thus obtaining a spectral content evolving with time.

Find an illustration of the idea here below.

<center> <img src="figs/sound2fv.svg" alt="" height="500"/> </center>

In [ ]:
Nft = 512  # Number of samples by FFT

# Homemade computation of stft
"Crop the signal such that its length is a multiple of Nft"
y = y[: L - L % Nft]
L = len(y)
"Reshape the signal with a piece for each row"
audiomat = np.reshape(y, (L // Nft, Nft))
audioham = audiomat * np.hamming(Nft)  # Windowing. Hamming, Hanning, Blackman,..
z = np.reshape(audioham, -1)  # y windowed by pieces
"FFT row by row"
stft = np.fft.fft(audioham, axis=1)
stft = np.abs(
    stft[:, : Nft // 2].T
)  # Taking only positive frequencies and computing the magnitude

"Library Librosa computing stft"
stft2 = librosa.stft(
    x, n_fft=Nft, hop_length=Nft, window="hamm", center="False"
)  # without downsampling the signal
stft4 = np.abs(librosa.stft(z, n_fft=Nft, hop_length=Nft, window="rect", center=False))

print(
    'Note: You can eventually add a "+1" in the "np.log" to get positive dB. This will look differently.'
)
print(L)

"Plots"
fig = plt.figure(figsize=(9, 3))
ax1 = fig.add_axes([0.0, 0.0, 0.42, 0.9])
ax2 = fig.add_axes([0.54, 0.0, 0.42, 0.9])

ax1.plot(np.arange(L) / fs_down, y, "b", label="Original")
ax1.plot(np.arange(L) / fs_down, z, "r", label="Hamming windowed by pieces")
ax1.set_xlabel("Time [s]")
ax1.legend()

plot_specgram(
    np.log(np.abs(stft2+1)),
    ax2,
    title="Specgram obtained with librosa.stft (full signal)",
    tf=len(x) / fs,
)
plt.show()

"Comparing the spectrograms"
fig2 = plt.figure(figsize=(12, 3))
ax3 = fig2.add_axes([0.0, 0.0, 0.28, 0.9])
ax4 = fig2.add_axes([0.34, 0.0, 0.28, 0.9])
ax5 = fig2.add_axes([0.68, 0.0, 0.28, 0.9])
plot_specgram(np.log(np.abs(stft)+1), ax3, title="Homemade specgram", tf=len(y) / fs_down) # +1 added
plot_specgram(
    np.log(np.abs(stft4)+1), # +1 added
    ax4,
    title="Specgram obtained with librosa.stft",
    tf=len(y) / fs_down,
)
plot_specgram(
    np.log(np.abs(stft - stft4[:-1, :])), ax5, title="Difference", tf=len(y) / fs_down
)
plt.show()

What differences can you notice between the upper spectrogram and the two others at the bottom?

<u> Remark:</u> The motivation for having a homemade version of the spectrogram is that in practice these computations will be embedded in the Nucleo board (the transmitter) and coded in C with fixed-point representation. Thus, as your classification model will be trained on feature vectors computed in Python but evaluated on feature vectors computed in C, it's crucial to know what are the exact steps followed for the signal transformation, to have these steps implementable in C, and to ensure they are the same.

## 3) From Hz to Melspectrogram

Now we have done the major part of the job. But recall that this information will have to be transmitted wirelessly from your circuit board (transmitter) to a base station (receiver). It is thus good practive to try synthetizing a bit the content of this spectrogram. <br>

A popular approach is to transform the frequency axis from Hz to Mel unit. The intuition behind this transformation is that the human ear will more easily distinguish between $100$ and $200$ Hz than between $3000$ and $3100$ Hz. So higher frequencies are more likely to be put together in very fewer coefficients. <br>

This last step will thus consist in replacing each column of the spectrogram ``stft`` with size $N_{FT}$ by a shorter column with size $N_{mel} \ll N_{FT}$. To do so, we will use an Hz to Mel (``Hz2Mel``) transformation matrix provided by ``librosa``, and apply a matrix multiplication for each column.

In [ ]:
Nmel = 20

"Obtain the Hz2Mel transformation matrix"
mels = librosa.filters.mel(sr=fs_down, n_fft=Nft, n_mels=Nmel)
mels = mels[:, :-1]

### TO COMPLETE
### Normalize the mels matrix such that its maximum value is one.
mels = mels

"Plot"
plt.figure(figsize=(5, 4))
plt.imshow(mels, aspect="auto", origin="lower")
plt.colorbar()
plt.title("Hz2Mel transformation matrix")
plt.xlabel("$N_{FT}$")
plt.ylabel("$N_{Mel}$")
plt.show()

In [ ]:
"Melspectrogram computation"
### TO COMPLETE
###  Perform the matrix multiplication between the Hz2Mel matrix and stft.
melspec = mels @ stft
print(melspec.size)
print(stft.size)

"Plot"
fig = plt.figure(figsize=(9, 3))
ax1 = fig.add_axes([0.0, 0.0, 0.42, 0.9])
ax2 = fig.add_axes([0.54, 0.0, 0.42, 0.9])
plot_specgram(np.log(np.abs(stft)), ax=ax1, title="Specgram", tf=len(y) / fs_down)
plot_specgram(
    np.log(np.abs(melspec)+1), # +1 added
    ax=ax2,
    is_mel=True,
    title="Melspecgram",
    tf=len(y) / fs_down,
)
plt.show()

Do these two spectrogram look similar? :) <br> 
What is the gain in the number of coefficients?

## 4) Creating black boxes

Now you have seen how to make the computations. <br>
A universal procedure consists in writing functions that will serve as working blocks and hide the computation details.
We can then gradually increase the abstraction. <br>

As any programmer should do, you are strongly encourage to ``fill your functions with a clear and concise docstring``. This will help you later this year when you will want to make improvements to some parts of your code. <br>

**Remark**: In any hands-on session you will work on during this semester, you will be provided with some already implemented code to exploit for processing and visualisation.
The implemented code will sometimes have been written by the teaching staff in the most readable and modular way possible. However, this may happen you want to add some features or simply modify some functions. Feel free to do so. In general, you can modify anything as you wish. Even the Python packages!

In [ ]:
def resample(x, M=4):
    """
    Applies a low pass filter then downsamples a signal.
    Args:
      x (numpy array): Signal to be downsampled
      M (int):         Downsample factor

    Returns:
      y (numpy array): Downsampled signal
    """
    y = signal.resample(x, int(len(x) / M))
    return y


def specgram(y, Nft=512):
    """
    Computes the spectrogram (frequency over time) of a signal. 
    Args:
      y (numpy array): Audio signal we want to extract its spectogram
      Nft (int): Number of samples by FFT

    Returns:
      stft (numpy matrix): Specgram matrix
    """

    ### TO COMPLETE
    stft = np.abs(librosa.stft(y, n_fft=Nft, hop_length=Nft, window="rect", center=False))
    return stft


def melspecgram(x, Nmel=20, Nft=512, fs=44100, fs_down=11025):
    """Computes the melspecgram (frequency over time) of a signal, with the Mel unit.

    Args:
      x (numpy array): Audio signal 
      Nmel (int): Downsample factor
      Nft (int): Number of samples by FFT
      fs (int): Sample frequency
      fs_down (int): Downsampling frequency

    Returns:
      melspec (numpy matrix): Melspecgram matrix
    """

    ### TO COMPLETE, using the functions resample() and specgram() defined above
    M = fs//fs_down
    y = resample(x,M)
    stft = specgram(y,Nft)
    mels = librosa.filters.mel(sr=fs_down, n_fft=Nft, n_mels=Nmel)
    #mels = mels[:, :-1]
    melspec = mels @ stft

    return melspec

## 5) Show us your skills
You are now encouraged to apply the functions you created above to sounds from at least 3 different classes. Observe their spectrograms and comment. Is it easy to differentiate sounds from the classes you chose?

In [ ]:
### TO COMPLETE
# %matplotlib tk

### Choose 3 sounds from different classes to observe how their mel spectrograms differ
sound1 = dataset["fire", 5] # fire_13
sound2 = dataset["birds", 5] # birds_13
sound3 = dataset["chainsaw", 0] # chainsaw_0
sound4 = dataset["handsaw",6] # handsaw_14
sound5 = dataset["helicopter",6] # helicopter_14


"Compute the melspecgrams"
x1, _ = sf.read(sound1)
x2, _ = sf.read(sound2)
x3, _ = sf.read(sound3)
x4, _ = sf.read(sound4)
x5, _ = sf.read(sound5)

melspec1 = melspecgram(x1)
melspec2 = melspecgram(x2)
melspec3 = melspecgram(x3)
melspec4 = melspecgram(x4)
melspec5 = melspecgram(x5)


print('Note: Notice that here we added the "+1" for the visualization!')

# "Plot"
# fig = plt.figure(figsize=(12, 3))
# ax1 = fig.add_axes([0.0, 0.0, 0.28, 0.9])
# ax2 = fig.add_axes([0.33, 0.0, 0.28, 0.9])
# ax3 = fig.add_axes([0.66, 0.0, 0.28, 0.9])
# plot_specgram(
#     melspec1, ax=ax1, is_mel=True, title=get_cls_from_path(sound1), tf=len(y) / fs_down
# )
# plot_specgram(
#     melspec2, ax=ax2, is_mel=True, title=get_cls_from_path(sound2), tf=len(y) / fs_down
# )
# plot_specgram(
#     melspec3, ax=ax3, is_mel=True, title=get_cls_from_path(sound3), tf=len(y) / fs_down
# )
# plt.show()

# "Plot"
# fig = plt.figure(figsize=(12, 3))
# ax4 = fig.add_axes([0.1, 0.0, 0.28, 0.9])
# ax5 = fig.add_axes([0.75, 0.0, 0.28, 0.9])
# plot_specgram(
#     melspec4, ax=ax4, is_mel=True, title=get_cls_from_path(sound4), tf=len(y) / fs_down
# )
# plot_specgram(
#     melspec5, ax=ax5, is_mel=True, title=get_cls_from_path(sound5), tf=len(y) / fs_down
# )

fig1 = plt.figure()
fig2 = plt.figure()
fig3 = plt.figure()
fig4 = plt.figure()
fig5 = plt.figure()
ax1 = fig1.add_subplot()
ax2 = fig2.add_subplot()
ax3 = fig3.add_subplot()
ax4 = fig4.add_subplot()
ax5 = fig5.add_subplot()

plot_specgram(
    melspec1, ax=ax1, is_mel=True, title=get_cls_from_path(sound1), tf=len(y) / fs_down
)
ax1.axvline(x=4.40, color='purple', linestyle='-')
ax1.axvline(x=4.9, color='purple', linestyle='-')

plot_specgram(
    melspec2, ax=ax2, is_mel=True, title=get_cls_from_path(sound2), tf=len(y) / fs_down
)
ax2.axvline(x=2.5, color='purple', linestyle='-')
ax2.axvline(x=3, color='purple', linestyle='-')

plot_specgram(
    melspec3, ax=ax3, is_mel=True, title=get_cls_from_path(sound3), tf=len(y) / fs_down
)
ax3.axvline(x=2.4, color='purple', linestyle='-')
ax3.axvline(x=2.9, color='purple', linestyle='-')

plot_specgram(
    melspec4, ax=ax4, is_mel=True, title=get_cls_from_path(sound4), tf=len(y) / fs_down
)
ax4.axvline(x=0.6, color='purple', linestyle='-')
ax4.axvline(x=1.1, color='purple', linestyle='-')

plot_specgram(
    melspec5, ax=ax5, is_mel=True, title=get_cls_from_path(sound5), tf=len(y) / fs_down
)
ax5.axvline(x=2.5, color='purple', linestyle='-')
ax5.axvline(x=3, color='purple', linestyle='-')

plt.show()

### Question:
Briefly comment what is intuitive for you in the content of these 3 spectrograms respectively with the corresponding classes.
How can you differentiate them?

### Part 1:
**Is your downsampling working properly? What can you observe on the spectrum of the downsampled signal? How is this phenomenon named and what is its origin?**

$\Rightarrow$  Yes, our downsampling is working properly, as for each M samples in the original signal, it only keeps one in the downsampled signal. We can observe that the downsample spectum doesn't match the original one. This phenomenon is called aliasing. It comes from the fact that as the signal is downsampled at a frequency $\frac{f_s}{M}$ and therefore its spectrum is repeated at each multiple of this frequency, thus creating interference betwen each occurence of the spectrum. Using a low-pass filter to limit the maximum frequency of the signal to fulfil Shannon-Nyquist criterion avoids aliasing and will be implemented.

**Can you hear the differences between the downsampled versions of the audio signal and the original one?** 

$\Rightarrow$ We hear that the downsampled version of the signal has its high frequency content removed as it has passed through a low pass filter. This can be seen on the Fourier Transform above. For some signals, we can detect a diminution in the intensity of the sound. This might be avoided by normalizing the signal intensity.

**You can also try sounds from different classes by running again the code above and changing the choice of the variable `sound`. Maybe you will notice there are two original and two downsampled signals shown. Can you guess why you have two signals? How to deal with this?**

$\Rightarrow$  Observing two signals is due to the fact that those can be stereo and not mono. We didn't observe it from the given files but it could happen during the project.

### Part 2:
**What differences can you notice between the upper spectrogram and the two others at the bottom?**

$\Rightarrow$  The upper one (spectrogram of the full signal) has a frequency span of $20$ $[kHz]$ while the two bottom ones only show up to around $5$ $[kHz]$. This is due to downsampling. As we sample at a frequency of  $11.025$ $[kHz]$, by the Shannon-Nyquist theorem, we can only expect the maximal frequency of the sound we sample to be half the sample frequency. 

### Part 3:
**Do these two spectrogram look similar? :)**

$\Rightarrow$  More or less, the position of the maximum stays similar but the intensity changes for some frequency. More importance is given to lower frequencies in the melspectrogram (i.e. it represents them better) as those are the frequencies that are the most distinguishable by human ear.

**What is the gain in the number of coefficients?**

$\Rightarrow$  $G = \frac{stft.size}{melspec.size} = 12.8$

### Part 5:
**Observe their spectrograms and comment. Is it easy to differentiate sounds from the classes you chose?**

Two main features of the spectrograms can help us differentiate the different classes of sound : 
 - Frequency content.
 - Frequecy distribution over time.

Certain sounds are quite easily differentiable, but some others present similar features and the classification might involved. Moreover, certain sounds from the same class present very different spectrograms, and sometimes multiple classes are present in a sound. 

**Briefly comment what is intuitive for you in the content of these 3 spectrograms respectively with the corresponding classes. How can you differentiate them?**

Fire : usually emits very low frequencies, no particular periodicity (spectrogram should look "constant"), and the frequencies are distributed among a rather limited interval. However, crackling sounds may induce higher frequencies (only in certain sounds of the dataset), which could be quite hard to classify during classification.

Birds : higher frequencies, around 15 mel. As the chirping of the bird is not continuous, we should expect "impulse" patterns, such as the one observed above.

Chainsaw : the frequency content is spread over a large frequency band with no particular distinguishable feature. It seems as though the sound of a chainsaw is filled with various frequencies.

Handsaw : just like for the chainsaw, the frequencies observed on the spectrogram are quite spread out, but we can clearly observe the periodicity of the movement actioning the handsaw. This kind of strict periodicity could be a clear sign in our classification methods.

Helicopter : as for the chainsaw and the handsaw, the frequencies are spread out, but the spectrogram contains more low frequency content as it will hardly ever exceed 10 mel